In [1]:
import numpy as np
import re
import datetime as dt
import sqlalchemy

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy.sql import exists  

from flask import Flask, jsonify


In [3]:
# Create engine and reflect database tables
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement
Station = Base.classes.station


In [4]:
# Create Flask app
app = Flask(__name__)


In [5]:
# Create routes
@app.route("/")
def welcome():
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/start (enter as YYYY-MM-DD)<br/>"
        f"/api/v1.0/start/end (enter as YYYY-MM-DD/YYYY-MM-DD)"
    )

@app.route("/api/v1.0/precipitation")
def precipitation():
    session = Session(engine)
    results = (session.query(Measurement.date, Measurement.tobs)
                      .order_by(Measurement.date))
    precipitation_date_tobs = [{"date": row.date, "tobs": row.tobs} for row in results]
    return jsonify(precipitation_date_tobs)

@app.route("/api/v1.0/stations")
def stations():
    session = Session(engine)
    results = session.query(Station.name).all()
    station_details = list(np.ravel(results))
    return jsonify(station_details)

@app.route("/api/v1.0/tobs")
def tobs():
    session = Session(engine)
    latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
    query_start_date = dt.datetime.strptime(latest_date[0], '%Y-%m-%d') - dt.timedelta(days=365)
    q_station_list = (session.query(Measurement.station, func.count(Measurement.station))
                             .group_by(Measurement.station)
                             .order_by(func.count(Measurement.station).desc())
                             .all())
    station_hno = q_station_list[0][0]
    results = (session.query(Measurement.station, Measurement.date, Measurement.tobs)
                      .filter(Measurement.date >= query_start_date)
                      .filter(Measurement.station == station_hno)
                      .all())
    tobs_list = [{"Date": row.date, "Station": row.station, "Temperature": row.tobs} for row in results]
    return jsonify(tobs_list)

@app.route("/api/v1.0/<start>") 
def start_only(start):
    session = Session(engine)
    date_range_max = session.query(Measurement.date).order_by(Measurement.date.desc()).first()[0]
    date_range_min = session.query(Measurement.date).first()[0]
    valid_entry = session.query(exists().where(Measurement.date == start)).scalar()
    if valid_entry:
    	results = (session.query(func.min(Measurement.tobs),
    				 func.avg(Measurement.tobs),
    				 func.max(Measurement.tobs))
    				  .filter(Measurement.date >= start).all())
    	tmin, tavg, tmax = results[0]
    	result_printout = (['Entered Start Date: ' + start,
    						'The lowest Temperature was: '  + str(tmin) + ' F',
    						'The average Temperature was: ' + str(round(tavg, 1)) + ' F',
    						'The highest Temperature was: ' + str(tmax) + ' F'])
    	return jsonify(result_printout)
    else:
    	return jsonify({"error": f"Measurement data not available for the date {start}. Please enter a valid date between {date_range_min} and {date_range_max}."})
